# ICHI multiplayer

#### This is the server and port variable, you need to change this to play on a specific server

In [64]:
# This is the IP address of the server you want to connect to
SERVER = "localhost"

# This is the port of the server you want to connect to
PORT = 8080

#### Import all library

In [65]:
# import of all dependencies needed for the project
import pygame
import socket
import time
import asyncio
import threading
from datetime import datetime, timedelta

# This library is used to communicate arrays between client and server
import pickle

#### Button class

Class that manage buttons and buttons events

Taken at :
https://github.com/russs123/pygame_tutorials/blob/main/Button/button.py

By russs123 : 
https://github.com/russs123

In [66]:
class Button():
    def __init__(self, x, y, image, scale):
        width = image.get_width()
        height = image.get_height()
        self.image = pygame.transform.scale(image, (int(width * scale), int(height * scale)))
        self.rect = self.image.get_rect()
        self.rect.topleft = (x, y)
        self.clicked = False

    def draw(self, surface):
        action = False
        
        # Get mouse position
        pos = pygame.mouse.get_pos()

        # Check mouseover and clicked conditions
        if self.rect.collidepoint(pos):
            if pygame.mouse.get_pressed()[0] == 1 and self.clicked == False:
                self.clicked = True
                action = True

        if pygame.mouse.get_pressed()[0] == 0:
            self.clicked = False

        # Draw button on screen
        surface.blit(self.image, (self.rect.x, self.rect.y))

        return action

#### Pygame initialisation

In [67]:
pygame.init()
screen = pygame.display.set_mode((1280,720))
clock = pygame.time.Clock()

logo = pygame.image.load('./assets/logo.png').convert_alpha()

pygame.display.set_icon(logo)

# This variable is used to get the state of the application (running or not)
run = True
baseFont = pygame.font.Font(None,32)
pygame.display.set_caption('ICHI - HE-Arc 2023 - By JATON David and NADDEO Eddy')

#### Pygame quit event
Function that manage the "quit" event (disconnect from server, quit application)

In [68]:
def quitEvent():
    run = False
    send(DISCONNECT_MESSAGE)
    pygame.quit()

#### Socket and var initialisation

In [69]:
# Server communication parameters
HEADER = 64
FORMAT = 'utf-8'

# Const for diconnect message
DISCONNECT_MESSAGE = ['!DISCONNECT']

# Server initialisation
ADDR = (SERVER, PORT)
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
error = False

# Trying the connection. If it failed, put the application in error (server unreachable)
try:
    client.connect(ADDR)
except:
    error = True
connected = True

# Varables that contain the message coming from the server
currentMessage = ["none","none","none"]
tmpMessage = ["none", "none", "none"]

# Variable that contain the room code
currentRoom = ""

# Variable for the username
username = ""

# List of all players in the room
players = []

# Hand of the player
myHand = []

# Heap of the current game
heap = []

# List of numbers, colors and special cards of ICHI (uno) game
listNumber = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,18,14,15,16,17,19,20,21,22]
listColor = [0, 1, 2, 3, 4]
listSpecialCard = [[1,4], [2,4], [3,4] , [4,4], [6,4] , [7,4] , [8,4] , [9,4]]
             
# Variable that indicate if it's the turn of the current player
myTurn = False

#### Socket loop

This function start a loop (will be useful to receive sockets messages)

In [70]:
def start_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

#### Socket get message

This function receive the server message and put it in the currentMessage (and tmpMessage if needed) variable

In [71]:
async def socket():
    # Global variable import
    global currentMessage
    global run
    global tmpMessage
    global myTurn
    
    # Infinit loop to receive messages
    while run:
        message = client.recv(512)
        currentMessage = pickle.loads(message)
        
        # If this is the turn of the current player, load tmpMessage with the current message currentMessage
        if currentMessage[len(currentMessage)-1] == "yourTurn":
            tmpMessage = currentMessage
            myTurn = True

#### Send function

Function that send in the correct format the message "msg" to the server

In [72]:
def send(msg):
    message = pickle.dumps(msg)
    msg_lenght = len(message)
    send_lenght = str(msg_lenght).encode(FORMAT)
    send_lenght += b' ' * (HEADER - len(send_lenght))
    client.send(send_lenght)
    client.send(pickle.dumps(msg))

#### Load assets

Cell to load all the assets of the game

In [73]:
newGameBtn = pygame.image.load('./assets/Buttons/New Game 192x48.png').convert_alpha()
quitBtn = pygame.image.load('./assets/Buttons/Quit 192x48.png').convert_alpha()
exitBtn = pygame.image.load('./assets/Buttons/Exit 192x48.png').convert_alpha()
newBtn = pygame.image.load('./assets/Buttons/New 192x48.png').convert_alpha()
joinBtn = pygame.image.load('./assets/Buttons/Load Game 192x48.png').convert_alpha()
quitGameBtn = pygame.image.load('./assets/Buttons/Quit Game 192x48.png').convert_alpha()
startGameBtn = pygame.image.load('./assets/Buttons/Start Game 192x48.png').convert_alpha()
creditBtn = pygame.image.load('./assets/Buttons/Credits 192x48.png').convert_alpha()
stack = pygame.image.load('./assets/Cards/Pile.png').convert_alpha()
spriteSheetCards = pygame.image.load('./assets/Cards/Cards.png').convert_alpha()
spriteSheetCardsGrayed = pygame.image.load('./assets/Cards/Cards.png').convert_alpha()
background = pygame.image.load('./assets/background.jpg').convert_alpha()
changeRed = pygame.image.load('./assets/Cards/ChangeRed.png').convert_alpha()
changeBlue = pygame.image.load('./assets/Cards/ChangeBlue.png').convert_alpha()
changeYellow = pygame.image.load('./assets/Cards/ChangeYellow.png').convert_alpha()
changeGreen = pygame.image.load('./assets/Cards/ChangeGreen.png').convert_alpha()
plus4Red = pygame.image.load('./assets/Cards/Plus4Red.png').convert_alpha()
plus4Blue = pygame.image.load('./assets/Cards/Plus4Blue.png').convert_alpha()
plus4Yellow = pygame.image.load('./assets/Cards/Plus4Yellow.png').convert_alpha()
plus4Green = pygame.image.load('./assets/Cards/Plus4Green.png').convert_alpha()
uno = pygame.image.load('./assets/Buttons/Uno 192x48.png').convert_alpha()
counterUno = pygame.image.load('./assets/Buttons/Counter Uno 192x48.png').convert_alpha()
arrowLeft = pygame.image.load('./assets/arrowLeft.png').convert_alpha()
arrowRight = pygame.image.load('./assets/arrowRight.png').convert_alpha()
hearc = pygame.image.load('./assets/hearcLogo.png').convert_alpha()
replayBtn = pygame.image.load('./assets/Buttons/Restart Game 192x48.png').convert_alpha()

#### Load image of file 

Function used to load sprites on a sprite-sheet

In [74]:
def get_image(sheet, frameW, frameH, width, height, scale):
        image = pygame.Surface((width, height)).convert_alpha()
        image.blit(sheet, (0, 0), ((frameW * width), ((frameH * height)), width, height))
        image = pygame.transform.scale(image, (width * scale, height * scale))
        return image

#### Text Display

This function is used for the input. It draw the text on the screen.

In [75]:
textCol = (255,255,255)
colorTextRect = (200,200,200)
def drawText(text, font, textCol, x, y):
    img = font.render(text, True, textCol)
    screen.blit(img, (x, y))

#### Join game menu

This is the function of the join menu. This menu contain all you need to create or join an existing room.

In [76]:
def joinMenu():
    
    # Global variable import
    global currentMessage
    global currentRoom
    global username

    # This variable contains the error message of the server if an error appears when you tried to join a room or create a room
    errorMessage = ""
    
    # This variable indicate if the current screen is "active". It's used to go back to previous screen for example (when set to False)
    localRun = True

    # This variable indicate if the user is able to click on the buttons. It's used to avoid "double" click (click on a button of the current screen AND of the previous screen)
    abilityToClick = False

    # Variables that contains screen size (x and y)
    x, y = pygame.display.get_surface().get_size()
    
    # Create rectangle as text box
    codeTextBox = pygame.Rect(x/2-((joinBtn.get_width()/2)*1.5),3*y/6-((newBtn.get_height()/2)*1.5)+ (newBtn.get_height()/2)-7, 105,joinBtn.get_height())
    usernameTextBox = pygame.Rect(x/2-((joinBtn.get_width()/2)*1.5),y/6-((newBtn.get_height()/2)*1.5)+ (newBtn.get_height()/2)-7, joinBtn.get_width()*1.5,joinBtn.get_height())
    
    # Variable for the text box (color of the rectangle and state of the box selected or not)
    activeCode = False
    activeUsername = False
    
    # String for the text box (room code in this case)
    code = ""
    
    # Definition of the buttons of the screen (x and y position are adaptative in case we want to change window size)
    joinRoom = Button(x/2-((joinBtn.get_width()/2)*1.5), 4*y/6-((joinBtn.get_height()/2)*1.5), joinBtn, 1.5)
    newRoom = Button(x/2-((newBtn.get_width()/2)*1.5), 2*y/6-((newBtn.get_height()/2)*1.5), newBtn, 1.5)
    exitMenu = Button(x/2-((exitBtn.get_width()/2)*1.5), 5*y/6-((exitBtn.get_height()/2)*1.5), exitBtn, 1.5)
        
    # Loop of the current screen
    while localRun:
        
        # Reset the screen by drawing background in front
        screen.blit(background, (0, 0))
        
        # Draw the text box
        pygame.draw.rect(screen,colorTextRect,codeTextBox,2)
        pygame.draw.rect(screen,colorTextRect,usernameTextBox,2)
        
        # Write text
        label = baseFont.render(errorMessage, 1, textCol)
        screen.blit(label, (x/2-((newBtn.get_width()/2)*1.5), y/9-((newBtn.get_height()/2)*1.5) + newBtn.get_height()/2))
        label = baseFont.render("Your username :", 1, textCol)
        screen.blit(label, (x/2-((newBtn.get_width()/2)*1.5) - 250, y/6-((newBtn.get_height()/2)*1.5) + newBtn.get_height()/2))
        label = baseFont.render("Room code :", 1, textCol)
        screen.blit(label, (x/2-((joinBtn.get_width()/2)*1.5) - 250, 3*y/6-((joinBtn.get_height()/2)*1.5) + joinBtn.get_height()/2))
        
        # Set the location or where we can write (room code and username input)
        drawText(username, baseFont, textCol, x/2-(newBtn.get_width()/2)*1.5 ,y/6-((newBtn.get_height()/2)*1.5)+ newBtn.get_height()/2)
        drawText(code, baseFont, textCol, x/2-(newBtn.get_width()/2)*1.5 ,3*y/6-((newBtn.get_height()/2)*1.5)+ newBtn.get_height()/2)
        
        # Looking for all pygame events
        for event in pygame.event.get():
            
            # Handle QUIT event
            if event.type == pygame.QUIT:
                quitEvent()
                
            # Check if a text box is selected
            if event.type == pygame.MOUSEBUTTONDOWN:
                if codeTextBox.collidepoint(event.pos):
                    activeCode = True
                else:
                    activeCode = False
                if usernameTextBox.collidepoint(event.pos): 
                    activeUsername = True
                else:
                    activeUsername = False
                    
            # Write in the code text box
            if activeCode == True:     
                if event.type == pygame.TEXTINPUT:
                    if len(code) < 5:
                        if event.text != "," and event.text != ".":
                            code += event.text.upper()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_BACKSPACE:
                        code = code[:-1]
                        
            # Write in the username text box            
            if activeUsername == True:     
                if event.type == pygame.TEXTINPUT:
                    if len(username) < 13:
                        if event.text != "," and event.text != ".":
                            username += event.text
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_BACKSPACE:
                        username = username[:-1]
                        
        # Check if newRoom button was pressed
        if newRoom.draw(screen) and abilityToClick:
            abilityToClick = False
            
            # Checking if the username is empty and if not, create a room
            if len(username) > 0:
                
                # Sending the "createRoom" message to the server
                send([f'{username}','createRoom',-1])
                
                # Waiting for the server response
                while currentMessage[0] != "error" or currentMessage[0] != "joinNow":
                    
                    # If the server returned an error, give it to the user
                    if currentMessage[0] == "error":
                        currentRoom = currentMessage[1]
                        errorMessage = "Cannot create a room."
                        break
                        
                    # Else if the response is "joinNow", open lobby menu
                    elif currentMessage[0] == "joinNow":
                        currentRoom = currentMessage[1]
                        errorMessage = ""
                        lobbyMenu()
                        break
                
                # Reset the current message to avoid infinit loop
                currentMessage = ["none","none","none"]
                
            else:
                errorMessage = "Please enter a username."
                
        # Check if joinRoom button was pressed
        if joinRoom.draw(screen) and abilityToClick:
            abilityToClick = False
            
            # Checking if the username and room code are not empty and if it's the case, create a room
            if len(username) > 0 and len(code) > 0:
                
                # Sending the "joinRoom" with the room code the user want to join to the server
                send([f'{username}','joinRoom',f'{code}'])
                
                # Waiting for the server response
                while currentMessage[0] != "error" or currentMessage[0] != "joinNow":
                    
                    # If the server returned an error, give it to the user
                    if currentMessage[0] == "error":
                        currentRoom = currentMessage[1]
                        errorMessage = "Cannot join this room (may be full, not exist or username already in use)."
                        break
                        
                    # Else if the response is "joinNow", open lobby menu
                    elif currentMessage[0] == "joinNow":
                        currentRoom = currentMessage[1]
                        errorMessage = ""
                        lobbyMenu()
                        break
                        
                # Reset the current message to avoid infinit loop
                currentMessage = ["none","none","none"]
                
            else:
                errorMessage = "Please enter a username and a room code."
                
        # Check if exitMenu button was pressed
        if exitMenu.draw(screen) and abilityToClick:
            abilityToClick = False
            
            # Close this screen
            localRun = False
            
        # Reseting the ability to click
        abilityToClick = True
        
        # Updating frame
        pygame.display.flip()
        

#### Current lobby menu

This is the function of the lobby menu. This menu contain list of players and ability to start the game if you are the room creator.

In [77]:
def lobbyMenu():

    # Global variable import
    global players
    global currentRoom
    global currentMessage
    global username
    global myHand
    global heap
    
    # This variable contains the error message of the server if an error appears when you tried to join a room or create a room
    errorMessage = ""
    
    # This variable indicate if the current screen is "active". It's used to go back to previous screen for example (when set to False)
    localRun = True
    
    # This variable indicate if the user is able to click on the buttons. It's used to avoid "double" click (click on a button of the current screen AND of the previous screen)
    abilityToClick = False

    # Variables that contains screen size (x and y)
    x, y = pygame.display.get_surface().get_size()
    
    # Definition of the button of the screen (x and y position are adaptative in case we want to change window size)
    quitGame = Button(x/2-((quitGameBtn.get_width()/2)*1.5), 2*y/3-((quitGameBtn.get_height()/2)*1.5), quitGameBtn, 1.5)
    startGame = Button(x/2-((startGameBtn.get_width()/2)*1.5), y/3-((startGameBtn.get_height()/2)*1.5), startGameBtn, 1.5)
    
    # Sending the "getPlayers" message to the server. This message is sent to ask the player list to the server
    send([f'{username}','getPlayers',f'{currentRoom}'])
    
    # Loop of the current screen
    while localRun:
        
        # Reset the screen by drawing background in front
        screen.blit(background, (0, 0))
        
        # Drawing the error message on the screen
        label = baseFont.render(errorMessage, 1, textCol)
        screen.blit(label, (x/2-((newBtn.get_width()/2)*1.5), y/9-((newBtn.get_height()/2)*1.5) + newBtn.get_height()/2))
        
        # If the message returned by the server is setPlayer...
        if currentMessage[0] == "setPlayers":
            
            # Update the local player list
            players = []
            counter = 0
            for msg in currentMessage:
                if counter > 0:
                    players.append(msg)
                counter = counter + 1
                
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
        
        # Drawing the current room text on the screen
        label = baseFont.render(f"Current room : {currentRoom}", 1, textCol)
        screen.blit(label, (50, 50))
        
        # If the message returned by the server is startNow...
        if currentMessage[0] == "startNow":
            abilityToClick = False
            
            # Updating player hand and heap to match with the server
            myHand = currentMessage[2]
            heap = currentMessage[1]
            
            # Open game table
            gameTable()
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
            
        # If the message returned by the server is notAValidNumberOfPerson...
        if currentMessage[0] == "notAValidNumberOfPerson":
            
            # Set the current error message to indicate the user there is an error
            errorMessage = "You can't start the game. There are maybe some players missing."
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
        
        counter = 0
        
        # Draw player list on the screen
        for p in players:
            counter = counter + 1
            label = baseFont.render(f"Player {counter} : {p}", 1, textCol)
            screen.blit(label, (50, 50*(counter+1)))
        
        # Checking startGame button event 
        if startGame.draw(screen) and abilityToClick:
            abilityToClick = False

            # Sending the "startGame" message to the server. This message is sent to ask the server to start the game
            send([f'{username}','startGame',f'{currentRoom}'])
        
        # Checking quitGame button event
        if quitGame.draw(screen) and abilityToClick:
            abilityToClick = False
            players = []
            
            # Sending the "startGame" message to the server. This message indicate to the server that the user left the room
            send([f'{username}','quitRoom',f'{currentRoom}'])
            
            # Close this screen
            localRun = False 
        
        # Reseting the ability to click
        abilityToClick = True
        
        # Updating frame
        pygame.display.flip()
        
        # Looking for all pygame events
        for event in pygame.event.get():
            
            # Handle QUIT event
            if event.type == pygame.QUIT:
                quitEvent()

#### End screen

This is the function of the end screen. This screen show up if a user left the room during a party, or if someone in the room won the game.

In [78]:
def endScreen(message, winner):
    
    # Global variable import
    global currentRoom
    global username
    global currentMessage
    global tmpMessage
    global players
    global myHand
    global heap
    
    # This variable contains the error message of the server if an error appears when you tried to join a room or create a room
    errorMessage = ""
    
    # This variable indicate if the current screen is "active". It's used to go back to previous screen for example (when set to False)
    localRun = True
    
    # This variable indicate if the user is able to click on the buttons. It's used to avoid "double" click (click on a button of the current screen AND of the previous screen)
    abilityToClick = False
    
    # Variables that contains screen size (x and y)
    x, y = pygame.display.get_surface().get_size()
    
    # Reset the current message to avoid propagation from previous message
    currentMessage = ["none","none","none"]
    
    # Definition of the buttons of the screen (x and y position are adaptative in case we want to change window size)
    newGame = Button(x/2-((newGameBtn.get_width()/2)*1.5), 4*y/6-((newGameBtn.get_height()/2)*1.5), newGameBtn, 1.5)
    quit = Button(x/2-((quitBtn.get_width()/2)*1.5), (y-quitBtn.get_height()-(quitBtn.get_height()/2))-20, quitBtn, 1.5)
    
    # Checking if the sceen has to be the one with restart button (end game with winner) or if it is the one with the "person left" message (end game with user lefting the room)
    if message != "disconnected" and players[0] == username:
        replay = Button(x/2-((replayBtn.get_width()/2)*1.5), 3*y/6-((replayBtn.get_height()/2)*1.5),replayBtn,1.5)        
    
    # Loop of the current screen
    while localRun:
        
        # Reset the screen by drawing background in front
        screen.blit(background, (0, 0))
        
        # If the message returned by the server is partyEnd...
        if currentMessage[0] == "partyEnd":
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
        
        # If the message returned by the server is updateHeap...
        if currentMessage[0] == "updateHeap":
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
        
        # If the temp message returned by the server is updateHeap...
        if tmpMessage[0] == "updateHeap":
            
            # Reset the current temp message to avoid propagation from previous message
            tmpMessage = ["none","none","none"]
        
        # If the message returned by the server is setPlayers...
        if currentMessage[0] == "setPlayers":
            
            # It means someone quit the room, so we have to return the end screen of the end game if someone left the room (to properly delete it on server side)
            endScreen("disconnected",username)
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
        
        # If the message returned by the server is startNow...
        if currentMessage[0] == "startNow":
            
            # Restart a game !
            
            abilityToClick = False
            myHand = currentMessage[2]
            heap = currentMessage[1]
            gameTable()
            currentMessage = ["none","none","none"]
            
        # If the message returned by the server is error...
        if currentMessage[0] == "error":
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
        
        # Checking if the sceen has to be the one with restart button (end game with winner) or if it is the one with the "person left" message (end game with user lefting the room)
        if message == "disconnected":
            label = baseFont.render(f"A player of your room disconnected. Room has been deleted.", 1, textCol)
            screen.blit(label, (x/2-label.get_rect().width/2, 2*y/6-((replayBtn.get_height()/2)*1.5)))
        else:
            label = baseFont.render(f"Winner of the room : {winner}", 1, textCol)
            replay = Button(x/2-((replayBtn.get_width()/2)*1.5), 3*y/6-((replayBtn.get_height()/2)*1.5),replayBtn,1.5)
            counter = 0
            screen.blit(label, (x/2-label.get_rect().width/2, 2*y/6-((replayBtn.get_height()/2)*1.5)))
            label = baseFont.render(f"Only the owner (Player 1) can restart the room.", 1, textCol)
            screen.blit(label, (x/2-label.get_rect().width/2, 2*y/6-((replayBtn.get_height()/2)*1.5)+20))
            
            # List players on screen
            for p in players:
                counter = counter + 1
                label = baseFont.render(f"Player {counter} : {p}", 1, textCol)
                screen.blit(label, (50, 50*(counter+1)))
        
        # Checking newGame button event
        if newGame.draw(screen) and abilityToClick:
            abilityToClick = False
            
            # Sending the "leaveRoom" message to the server. This message indicate to the server that the user left the room
            send([f'{username}','leaveRoom',f'{currentRoom}'])
            
            # Open mainMenu
            mainMenu()
            
        # Checking quit button event
        if quit.draw(screen) and abilityToClick:
            abilityToClick = False
            
            # Quit game with quitEvent() function
            quitEvent()
            
        # Checking replay button event (only if you are in the "winned" end game screen)
        if message != "disconnected" and players[0] == username:
            if replay.draw(screen) and abilityToClick:
                abilityToClick = False
                
                 # Sending the "startGame" message to the server. This message is sent to ask the server to start the game
                send([f'{username}','startGame',f'{currentRoom}'])
        
        # Reseting the ability to click
        abilityToClick = True
        
        # Updating frame
        pygame.display.flip()
        
        # Looking for all pygame events
        for event in pygame.event.get():
            
            # Handle QUIT event
            if event.type == pygame.QUIT:
                quitEvent()

#### Credits screen

This is the function of the credit screen. This screen contain the credits of the game.

In [79]:
def creditScreen():
    
    # Global variable import
    global currentMessage
    
    # This variable indicate if the current screen is "active". It's used to go back to previous screen for example (when set to False)
    localRun = True
    
    # This variable indicate if the user is able to click on the buttons. It's used to avoid "double" click (click on a button of the current screen AND of the previous screen)
    abilityToClick = False
    
    # Variables that contains screen size (x and y)
    x, y = pygame.display.get_surface().get_size()
    
    # Reset the current message to avoid propagation from previous message
    currentMessage = ["none","none","none"]
    
    # Definition of the button of the screen (x and y position are adaptative in case we want to change window size)
    exit = Button(x/2-((exitBtn.get_width()/2)*1.5), (y-exitBtn.get_height()-(exitBtn.get_height()/2))-20, exitBtn, 1.5)
    
    # Loop of the current screen
    while localRun:
        
        # Reset the screen by drawing background in front
        screen.blit(background, (0, 0))
        
        # Drawing credits
        label = baseFont.render("Cards designs : mehrasaur (OpenGameArt)", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,10))
        label = baseFont.render("Buttons : russs123 (github)", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,60))
        label = baseFont.render("Buttons assets : Screaming Brain Studios (OpenGameArt)", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,110))
        label = baseFont.render("Other assets : pixabay.com / pngall.com", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,160))
        label = baseFont.render("Server-side and server-client communication : Naddeo Eddy", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,210))
        label = baseFont.render("(nb: communication logic adapted for notebooks and arrays from Tech With Tim logic, youtube)", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,260))
        label = baseFont.render("Client side cards logic : Jaton David", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,310))
        label = baseFont.render("Client side UX : Jaton David and Naddeo Eddy", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,360))
        screen.blit(pygame.transform.scale(hearc,(849,159)),(x/2 - 849/2,410))
        
        # Check if exit button was pressed
        if exit.draw(screen) and abilityToClick:
            abilityToClick = False
            
            # Close this screen
            localRun = False
            
        # Updating frame
        pygame.display.flip() 
        
        # Looking for all pygame events
        for event in pygame.event.get():
            
            # Handle QUIT event
            if event.type == pygame.QUIT:
                quitEvent()
                
        # Reseting the ability to click
        abilityToClick = True

#### Game table

In [80]:
def gameTable():
    
    # Global variable import
    global currentMessage
    global tmpMessage
    global currentRoom
    global listNumber
    global listColor
    global myHand
    global heap
    global players
    global username
    global myTurn
    
    # On gameTable initialisation, checking if the client is the first one to play
    if players[0] != username:
        myTurn = False
    
    # This variable indicate if the current screen is "active". It's used to go back to previous screen for example (when set to False)
    localRun = True
    
    # This variable indicate if the user is able to click on the buttons. It's used to avoid "double" click (click on a button of the current screen AND of the previous screen)
    abilityToClick = False
    
    # Variables that contains screen size (x and y)
    x, y = pygame.display.get_surface().get_size()
    
    # Reset the current message to avoid propagation from previous message
    currentMessage = ["none","none","none"]
    
    # Variable indicating if you are changing color
    changeColor = False
    
    # Variable indicating if you are changing color with +4 card
    plus4 = False
    
    # Getting image with sprite-sheet
    cardTmpInit = get_image(spriteSheetCards, listNumber[heap[0]] , listColor[heap[1]], 32, 48,3)
    
    # Variables containing cards size
    cardX = cardTmpInit.get_width()
    cardY = cardTmpInit.get_height()
    stackX = x/2-((cardX/2)-(cardX))
    stackY = y/2-(cardY/2)
    heapX = x/2-((cardX/2)+(cardX))
    heapY = y/2-(cardY/2)
    
    # Definition of the stack button of the screen (x and y position are adaptative in case we want to change window size)
    stackButton = Button(stackX,stackY,stack,3)
    
    # Definition of the changing color and +4 buttons of the screen (x and y position are adaptative in case we want to change window size)
    changeRedBtn    = Button(x/2-cardX/2-((cardX/2)+5), y/2-cardY/2-((cardY/2)+5), changeRed, 3)
    changeBlueBtn   = Button(x/2-cardX/2-((cardX/2)+5), y/2-cardY/2+((cardY/2)+5), changeBlue, 3)
    changeYellowBtn = Button(x/2-cardX/2+((cardX/2)+5), y/2-cardY/2-((cardY/2)+5), changeYellow, 3)
    changeGreenBtn  = Button(x/2-cardX/2+((cardX/2)+5), y/2-cardY/2+((cardY/2)+5), changeGreen, 3)
    plus4RedBtn    = Button(x/2-cardX/2-((cardX/2)+5), y/2-cardY/2-((cardY/2)+5), plus4Red, 3)
    plus4BlueBtn   = Button(x/2-cardX/2-((cardX/2)+5), y/2-cardY/2+((cardY/2)+5), plus4Blue, 3)
    plus4YellowBtn = Button(x/2-cardX/2+((cardX/2)+5), y/2-cardY/2-((cardY/2)+5), plus4Yellow, 3)
    plus4GreenBtn  = Button(x/2-cardX/2+((cardX/2)+5), y/2-cardY/2+((cardY/2)+5), plus4Green, 3)
    
    # Definition of the uno and counter uno buttons of the screen (x and y position are adaptative in case we want to change window size)
    unoBtn = Button((5*x/6)+(x/12)-(uno.get_width()/2), y-(2*uno.get_height())-(uno.get_height()/2), uno,1)
    counterUnoBtn = Button((5*x/6)+(x/12)-(uno.get_width()/2), y-uno.get_height()-(uno.get_height()/2),counterUno,1)
    
    # Table containing numbers of cards of all players in the room
    numberOfCards = []
    
    # Table containing timestamp of last card placed by users of all players in the room
    numberOfCardsTimeStamp = ["","","",""]
    
    # Table containing "uno" status of players
    inUno = [False, False, False, False]
    
    # Getting through all players in the room and sort them in the correct order in playersOrder
    throughPlayers = True
    counter = 0
    currentIndex = 0
    playersOrder = []
    while throughPlayers:
        if counter == len(players) - 1:
            throughPlayers = False
        if currentIndex == len(players):
            currentIndex = 0
            playersOrder.append(players[currentIndex])
            counter = counter + 1
            currentIndex = currentIndex + 1
        else:
            playersOrder.append(players[currentIndex])
            counter = counter + 1
            currentIndex = currentIndex + 1
    
    # Variable that contain the username of the player playing right now
    currentPlayerPlaying = playersOrder[0]

    # Setting numbers of cards of players to 7 (by default)
    for player in players:
        numberOfCards.append(7)
        
    # This variable indicate the way of the room. By default, way is "False"
    currentWay = False
    
    # Variable of the color of the player label (by default -> white, when playing -> green)
    playerColor = (255,255,255)

    # Loop of the current screen
    while localRun:
        
        # Reset placable state of cards
        notPlacable = False
        
        # Variable containing mouse position
        mousePosition = pygame.mouse.get_pos()
        
        # Reset the screen by drawing background in front
        screen.blit(background, (0, 0))
        
        # Table containing rectangles used for collision (click detection on a card)
        rectangles = []
        
        # If it's not my turn
        if not myTurn :
            
            # Draw all cards and rectangles normally (x and y position are adaptative in case we want to change window size)
            for i in range (0, len(myHand)):
                screen.blit(get_image(spriteSheetCards, listNumber[myHand[i][0]] , listColor[myHand[i][1]], 32, 48,3), (((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10))
                if (cardX * len(myHand)) < (x/6)*4:
                    rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                    rectangles.append(rect)
                    pygame.draw.rect(screen, (0,0,0), rect,  2)
                elif i != len(myHand)-1:
                    rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, (((x/6)*4))/len(myHand), cardY)
                    rectangles.append(rect)
                    pygame.draw.rect(screen, (0,0,0), rect,  2)
                else:
                    rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                    rectangles.append(rect)
                    pygame.draw.rect(screen, (0,0,0), rect,  2)
        
        # Else, if it's my turn
        else:
            
            # Draw all cards and rectangles and if I can't place the card that is drawed, make it gray (x and y position are adaptative in case we want to change window size)
            for i in range (0, len(myHand)):
                if heap[1] != 4 and myHand[i][0] == heap[0] or myHand[i][1] == heap[1] or myHand[i][1] == 4 or \
                (heap == [1, 4] and myHand[i][1] == 0 and plus4 == False and changeColor == False) or \
                (heap == [2, 4] and myHand[i][1] == 1 and plus4 == False and changeColor == False) or \
                (heap == [3, 4] and myHand[i][1] == 2 and plus4 == False and changeColor == False) or \
                (heap == [4, 4] and myHand[i][1] == 3 and plus4 == False and changeColor == False) or \
                (heap == [6, 4] and myHand[i][1] == 0 and plus4 == False and changeColor == False) or \
                (heap == [7, 4] and myHand[i][1] == 1 and plus4 == False and changeColor == False) or \
                (heap == [8, 4] and myHand[i][1] == 2 and plus4 == False and changeColor == False) or \
                (heap == [9, 4] and myHand[i][1] == 3 and plus4 == False and changeColor == False) or \
                plus4 == True or changeColor == True:
                    screen.blit(get_image(spriteSheetCards, listNumber[myHand[i][0]] , listColor[myHand[i][1]], 32, 48,3), (((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10))
                    if (cardX * len(myHand)) < (x/6)*4:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
                    elif i != len(myHand)-1:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, (((x/6)*4))/len(myHand), cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
                    else:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
                else :
                    pygame.transform.grayscale(spriteSheetCards, spriteSheetCardsGrayed)
                    screen.blit(get_image(spriteSheetCardsGrayed, listNumber[myHand[i][0]] , listColor[myHand[i][1]], 32, 48,3), (((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10))
                    if (cardX * len(myHand)) < (x/6)*4:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
                    elif i != len(myHand)-1:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, (((x/6)*4))/len(myHand), cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
                    else:
                        rect = pygame.Rect(((i+1)*(x/6)*4)/(len(myHand)+1)-(cardX/2)+(x/6), y-cardY-10, cardX, cardY)
                        rectangles.append(rect)
                        pygame.draw.rect(screen, (0,0,0), rect,  2)
        
        # If the message returned by the server is setPlayers...
        if currentMessage[0] == "setPlayers":
            
            # It means someone quit the room, so we have to return the end screen of the end game if someone left the room (to properly delete it on server side)
            endScreen("disconnected",username)
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
            
        # If the message returned by the server is takeCards...
        if currentMessage[len(currentMessage)-1] == "takeCards":
            
            # Set the current player playing (to draw it in green)
            currentPlayerPlaying = currentMessage[4]
            
            # Update player hand
            for cards in currentMessage[5]:
                myHand.append(cards)
                
            # Sending the "updateNumberOfCards" message to the server. This message is sent to update number of cards of this player for all other players in the room
            send([f'{username}','updateNumberOfCards',f'{currentRoom}',len(myHand)])
        
        # If the message returned by the server is startNow...
        if currentMessage[0] == "startNow":
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
        
        # If it's my turn, draw it on the screen
        if myTurn:
            label = baseFont.render(f"It's your turn !", 1, textCol)
            labelX = label.get_width()
            screen.blit(label, (x/2 - labelX/2, y/2 - cardY - 50))
        
        # If the message returned by the server is updateHeap...
        if currentMessage[0] == "updateHeap":
            
            # Set the current player playing (to draw it in green)
            currentPlayerPlaying = currentMessage[4]
            
            # Update the heap
            heap=currentMessage[1]
            
            # Update number of cards of last player played
            numberOfCards[playersOrder.index(currentMessage[2])] = currentMessage[3]
            
            # Checking if the player that just played has more than one card. If yes, put is "inUno" status to "False"
            if currentMessage[3] > 1:
                inUno[playersOrder.index(currentMessage[2])] = False
                
            # Updating timestamp of the player that just played
            numberOfCardsTimeStamp[playersOrder.index(currentMessage[2])] = datetime.now()
            
            # Checking if the last card is a "Changement de sens". If it's the case, change the way of the room
            if currentMessage[1] == [11,0] or currentMessage[1] == [11,1] or currentMessage[1] == [11,2] or currentMessage[1] == [11,3]:
                if currentWay == False:
                    currentWay = True
                else:
                    currentWay = False
                    
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
        
        # If the message returned by the server is updateHeap...
        if tmpMessage[0] == "updateHeap":
            
            # Set the current player playing (to draw it in green)
            currentPlayerPlaying = tmpMessage[4]
            
            # Update the heap
            heap=tmpMessage[1]
            
            # Update number of cards of last player played
            numberOfCards[playersOrder.index(tmpMessage[2])] = tmpMessage[3]
            
            # Checking if the player that just played has more than one card. If yes, put is "inUno" status to "False"
            if tmpMessage[3] > 1:
                inUno[playersOrder.index(tmpMessage[2])] = False
                
            # Updating timestamp of the player that just played
            numberOfCardsTimeStamp[playersOrder.index(tmpMessage[2])] = datetime.now()
            
            # Reset the current temp message to avoid propagation from previous message
            tmpMessage = ["none","none","none"]
            
        # If the message returned by the server is newCard...
        if currentMessage[0] == "newCard":
            
            # Update current player playing
            currentPlayerPlaying = currentMessage[4]
            
            # Update the hand of the playder
            myHand.append(currentMessage[3])
            
            # Update number of cards of the player
            numberOfCards[playersOrder.index(currentMessage[1])] = currentMessage[2]
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
            
        # If the message returned by the server is updateNumberOfCards...
        if currentMessage[0] == "updateNumberOfCards":
            
            # Index of the player we have to update the numbers of cards
            indexPlayerToUpdate = playersOrder.index(currentMessage[1])
            
            # Updating number of cards
            numberOfCards[playersOrder.index(currentMessage[1])] = currentMessage[2]
            
            # Checking if the player that just played has more than one card. If yes, put is "inUno" status to "False"
            if currentMessage[2] > 1:
                inUno[playersOrder.index(currentMessage[1])] = False
                
            # Updating timestamp of the player that just played
            numberOfCardsTimeStamp[playersOrder.index(currentMessage[1])] = datetime.now()
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
        
        # If the message returned by the server is partyEnd...
        if currentMessage[0] == "partyEnd":
            
            # Show end screen
            endScreen("winner",currentMessage[1])
            
        # If the message returned by the server is askedCard...
        if currentMessage[0] == "askedCard":
            
            # Update the current player playing
            currentPlayerPlaying = currentMessage[3]
            
            # Updating number of cards
            numberOfCards[playersOrder.index(currentMessage[1])] = currentMessage[2]
            
            # Checking if the player that just played has more than one card. If yes, put is "inUno" status to "False"
            if currentMessage[2] > 1:
                inUno[playersOrder.index(currentMessage[1])] = False
                
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
            
        # If the message returned by the server is takeCardsCounterUnoHit...
        if currentMessage[0] == "takeCardsCounterUnoHit":
            
            # Update the hand of the user
            for cards in currentMessage[1]:
                myHand.append(cards)
                
            # Sending the update to the server
            send([f'{username}','updateNumberOfCards',f'{currentRoom}',len(myHand)])
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
            
        # If the message returned by the server is unoFromPlayer...
        if currentMessage[0] == "unoFromPlayer":
            
            # Update inUno state of the player
            inUno[playersOrder.index(currentMessage[1])] = True
            
            # Reset the current message to avoid propagation from previous message
            currentMessage = ["none","none","none"]
        
        # Variable that contain the current index of the user
        currentIndex = players.index(username)
        
        # Go thgough all players and sort them in the correct order
        throughPlayers = True
        counter = 0
        playersOrder = []
        while throughPlayers:
            if counter == len(players) - 1:
                throughPlayers = False
            if currentIndex == len(players):
                currentIndex = 0
                playersOrder.append(players[currentIndex])
                counter = counter + 1
                currentIndex = currentIndex + 1
            else:
                playersOrder.append(players[currentIndex])
                counter = counter + 1
                currentIndex = currentIndex + 1
        
        # Change inUno state of the user if number of card is bigger than 1
        if len(myHand) > 1:
            inUno[playersOrder.index(username)] = False
            
        # Drawing players cards and name, and if they said ICHI
        for i in range(0, len(players)):
            if currentPlayerPlaying == playersOrder[i]:
                playerColor = (0,255,0)
            else:
                playerColor = (255,255,255)
            if i == 1:
                for j in range(0,numberOfCards[i]):
                    screen.blit(pygame.transform.rotate(get_image(spriteSheetCards, listNumber[10], listColor[4], 32, 48,3),270), (-cardY/2+30,((j+1)*(y/6)*4)/(numberOfCards[i]+1)-(cardX/2)+(y/6)))
                if inUno[i] == True:
                    label = baseFont.render(f"{playersOrder[i]} - ICHI !", 1, playerColor)
                else:
                    label = baseFont.render(f"{playersOrder[i]}", 1, playerColor)
                screen.blit(pygame.transform.rotate(label,270), (cardY-30, y/2 - label.get_rect().width/2))
            elif i == 2:
                for j in range(0,numberOfCards[i]):
                    screen.blit(pygame.transform.rotate(get_image(spriteSheetCards, listNumber[10], listColor[4], 32, 48,3),180), (((j+1)*(x/6)*4)/(numberOfCards[i]+1)-(cardX/2)+(x/6), -cardY/2+30))
                if inUno[i] == True:
                    label = baseFont.render(f"{playersOrder[i]} - ICHI !", 1, playerColor)
                else:
                    label = baseFont.render(f"{playersOrder[i]}", 1, playerColor)
                screen.blit(label, (x/2 - label.get_rect().width/2, cardY-30))
            elif i == 3:
                for j in range(0,numberOfCards[i]):
                    screen.blit(pygame.transform.rotate(get_image(spriteSheetCards, listNumber[10], listColor[4], 32, 48,3),90), (x-cardY/2-30,((j+1)*(y/6)*4)/(numberOfCards[i]+1)-(cardX/2)+(y/6)))
                if inUno[i] == True:
                    label = baseFont.render(f"{playersOrder[i]} - ICHI !", 1, playerColor)
                else:
                    label = baseFont.render(f"{playersOrder[i]}", 1, playerColor)
                screen.blit(pygame.transform.rotate(label,90), (x-cardY+5, y/2 - label.get_rect().width/2))
        
        # Size of arrow
        arrowX = arrowLeft.get_rect().width
        arrowY = arrowLeft.get_rect().height
        
        # Draw arrow in the current way of the room
        if currentWay == False:
            screen.blit(pygame.transform.scale(arrowLeft, (100, 100)), ((x/6)/2-50,y-cardY/2-60))
        else:
            screen.blit(pygame.transform.scale(arrowRight, (100, 100)), ((x/6)/2-50,y-cardY/2-60))
        
        # Draw heap
        screen.blit(get_image(spriteSheetCards, listNumber[heap[0]] , listColor[heap[1]], 32, 48,3), (heapX, heapY))
        pygame.draw.rect(screen, ((0,0,0)), pygame.Rect(heapX, heapY, cardX, cardY),  2)
        
        # Detect stackButton click and ask a new card to the server
        if stackButton.draw(screen) and abilityToClick and myTurn and plus4 == False and changeColor == False:
            abilityToClick = False
            myTurn = False
            send([f'{username}','demandingCard',f'{currentRoom}',len(myHand)+1])
        abilityToClick = True
        
        # Send to the server uno message if current size of the user hand is 1 (and if he pressed the ICHI button)
        if len(myHand) == 1 and inUno[playersOrder.index(username)] == False and plus4 == False and changeColor == False:
            if unoBtn.draw(screen) : 
                send([f'{username}','uno',f'{currentRoom}'])
        clickable = True
        
        # Check the counter ICHI event (when pressed on counter ICHI), and sending to server the correct information
        for player in playersOrder:
                if player != username and inUno[playersOrder.index(player)] == False and clickable and numberOfCards[playersOrder.index(player)] <= 1 and numberOfCardsTimeStamp[playersOrder.index(player)] + timedelta(seconds=5) < datetime.now():
                    if counterUnoBtn.draw(screen):
                        asValidCounterUno = False
                        for player in playersOrder:
                            if inUno[playersOrder.index(player)] == False and clickable and numberOfCards[playersOrder.index(player)] <= 1 and numberOfCardsTimeStamp[playersOrder.index(player)] + timedelta(seconds=5) < datetime.now():
                                asValidCounterUno = True
                                clickable = False
                                send([f'{username}','counterUno',f'{currentRoom}',f'{player}'])
                        if asValidCounterUno == False:
                            send([f'{username}','takeCardsCounterUno',f'{currentRoom}'])
        
        # When change color or +4 selected, draw the buttons to chose the color, and if pressed -> update heap
        if changeColor == True:
            if changeRedBtn.draw(screen) :
                myTurn = False
                changeColor = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[0],len(myHand)])
            elif changeBlueBtn.draw(screen) :
                myTurn = False
                changeColor = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[1],len(myHand)])
            elif changeYellowBtn.draw(screen) :
                myTurn = False
                changeColor = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[2],len(myHand)])
            elif changeGreenBtn.draw(screen) :
                myTurn = False
                changeColor = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[3],len(myHand)])        
        if plus4 == True :
            if plus4RedBtn.draw(screen) :
                myTurn = False
                plus4 = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[4],len(myHand)])
            elif plus4BlueBtn.draw(screen) :
                myTurn = False
                plus4 = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[5],len(myHand)])
            elif plus4YellowBtn.draw(screen) :
                myTurn = False
                plus4 = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[6],len(myHand)])
            elif plus4GreenBtn.draw(screen) :
                myTurn = False
                plus4 = False
                send([f'{username}','placing',f'{currentRoom}',listSpecialCard[7],len(myHand)])
                
        # Updating frame
        pygame.display.flip()
        
        # Looking for all pygame events
        for event in pygame.event.get():
            
            # Handle mouse button click event (on cards), check if card is placable (rules) and place it if it's the case
            if event.type == pygame.MOUSEBUTTONDOWN and changeColor == False and plus4 == False:
                if pygame.mouse.get_pressed()[0]:
                    for i in range (0, len(myHand)):
                        if rectangles[i].collidepoint(mousePosition) and myTurn:
                            if heap[1] != 4 and myHand[i][0] == heap[0] and myHand[i][1] != 4:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif myHand[i][1] == heap[1] and heap[1] != 4:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif myHand[i][1] == 4:
                                if myHand[i][0] == 0:
                                    changeColor = True
                                    myHand.pop(i)
                                elif myHand[i][0] == 5:
                                    plus4 = True
                                    myHand.pop(i)
                                    
                            # Special case for +4 and change color cards on the heap, beacause logic of listNumber and listColor is different for those cards
                            elif plus4 == False and changeColor == False and heap == [1, 4] and myHand[i][1] == 0:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif plus4 == False and changeColor == False and heap == [2, 4] and myHand[i][1] == 1:                               
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif plus4 == False and changeColor == False and heap == [3, 4] and myHand[i][1] == 2:                     
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif plus4 == False and changeColor == False and heap == [4, 4] and myHand[i][1] == 3:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            elif plus4 == False and changeColor == False and heap == [6, 4] and myHand[i][1] == 0:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)  
                            elif plus4 == False and changeColor == False and heap == [7, 4] and myHand[i][1] == 1:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)  
                            elif plus4 == False and changeColor == False and heap == [8, 4] and myHand[i][1] == 2:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)  
                            elif plus4 == False and changeColor == False and heap == [9, 4] and myHand[i][1] == 3:
                                myTurn = False
                                send([f'{username}','placing',f'{currentRoom}',myHand[i],len(myHand)-1])
                                myHand.pop(i)
                            else:
                                notPlacable = True
                            if len(myHand) == 0:
                                send([f'{username}','winning',f'{currentRoom}'])
                                endScreen("winner",username)
                                break
            
            # Handle QUIT event
            if event.type == pygame.QUIT:
                quitEvent()

#### Server error screen

This is the function of the error screen. This screen show up when server is unreachable.

In [81]:
def errorScreen():
    
    # This variable indicate if the current screen is "active". It's used to go back to previous screen for example (when set to False)
    localRun = True
    
    # Variables that contains screen size (x and y)
    x, y = pygame.display.get_surface().get_size()

    # Loop of the current screen
    while localRun:
        
        # Reset the screen by drawing background in front
        screen.blit(background, (0, 0))
        
        # Drawing the error message on the screen
        label = baseFont.render(f"The server \"{SERVER}\" with port \"{PORT}\" is not reachable.", 1, textCol)
        screen.blit(label, (x/2 - label.get_rect().width/2,y/2 - label.get_rect().height/2))
        
        # Updating frame
        pygame.display.flip()
        
        # Looking for all pygame events
        for event in pygame.event.get():
            
            # Handle QUIT event
            if event.type == pygame.QUIT:
                pygame.quit()
                
        # Reseting the ability to click
        abilityToClick = True

#### Pygame main menu

This is the function of the main menu.

In [82]:
def mainMenu():
    
    # Global variable import
    global run
    
    # This variable indicate if the user is able to click on the buttons. It's used to avoid "double" click (click on a button of the current screen AND of the previous screen)
    abilityToClick = False

    # Variables that contains screen size (x and y)
    x, y = pygame.display.get_surface().get_size()
    
    # Definition of the buttons of the screen (x and y position are adaptative in case we want to change window size)
    newGame = Button(x/2-((newGameBtn.get_width()/2)*1.5), y/4-((newGameBtn.get_height()/2)*1.5), newGameBtn, 1.5)
    credits = Button(x/2-((creditBtn.get_width()/2)*1.5), y/2-((creditBtn.get_height()/2)*1.5), creditBtn, 1.5)
    quit = Button(x/2-((quitBtn.get_width()/2)*1.5), 3*y/4-((quitBtn.get_height()/2)*1.5), quitBtn, 1.5)

    # Loop of the current screen - is the loop of the whole game too
    while run:
        
        # Reset the screen by drawing background in front
        screen.blit(background, (0, 0))
        
        # Checking newGame button event
        if newGame.draw(screen) and abilityToClick:
            abilityToClick = False
            
            # Open joinMenu
            joinMenu()
            
        # Checking credits button event
        if credits.draw(screen) and abilityToClick:
            abilityToClick = False
            
            # Open creditScreen
            creditScreen()
            
        # Checking quit button event
        if quit.draw(screen) and abilityToClick:
            abilityToClick = False
            
            # Quit game with quitEvent() function
            quitEvent()
            
        # Reseting the ability to click
        abilityToClick = True
        
        # Updating frame
        pygame.display.flip()
        
        # Looking for all pygame events
        for event in pygame.event.get():
            
            # Handle QUIT event
            if event.type == pygame.QUIT:
                quitEvent()
    

#### Start listening on socket

In [83]:
new_loop = asyncio.new_event_loop()
t = threading.Thread(target=start_loop, args=(new_loop,))
t.start()

asyncio.run_coroutine_threadsafe(socket(), new_loop)

<Future at 0x1ba6ed0e070 state=pending>

#### Pygame main loop

In [84]:
# Main loop of the game
while run:

    # Reset the screen by drawing background in front
    screen.blit(background, (0, 0))
    
    # Check if there is an error...
    if error == True:
        
        # Open errorScreen
        errorScreen()
        
    else:
        
        # Open mainMenu
        mainMenu()

    # Updating frame
    pygame.display.flip()
    
    # Looking for all pygame events
    for event in pygame.event.get():
        
        # Handle QUIT event
        if event.type == pygame.QUIT:
            run = False
            
# Handle QUIT event if the last line of this programme is triggered
quitEvent()

error: video system not initialized